# Setup

In [1]:
!pip install -q openai langchain chromadb faiss-cpu pypdf pdfplumber chromadb tiktoken docarray huggingface_hub sentence-transformers

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
tables 3.8.0 requires cython>=0.29.21, which is not installed.
llama-index 0.9.0 requires urllib3<2, but you have urllib3 2.2.1 which is incompatible.
botocore 1.29.76 requires urllib3<1.27,>=1.25.4, but you have urllib3 2.2.1 which is incompatible.
python-lsp-black 1.2.1 requires black>=22.3.0, but you have black 0.0 which is incompatible.


# Environment setup

In [2]:
!pip install -q openai langchain

In [1]:
import os
import openai

In [2]:
import pdfplumber
from pathlib import Path
import pandas as pd
from operator import itemgetter
import json
import tiktoken


In [19]:
# Set the API key
filepath ="D:/UpgradSecretKey.txt.txt"

with open(filepath, "r") as f:
  openai.api_key = ' '.join(f.readlines())

In [4]:
myntraDataSet="processedFashionData.csv"

In [5]:
import pandas as pd;

df=pd.read_csv(myntraDataSet)

df.head()

,Unnamed: 0.1,Unnamed: 0,p_id,name,price,colour,brand,img,ratingCount,avg_rating,description,p_attributes,Metadata,Text_Length
0,5975,35,12221918.0,SASSAFRAS Women Black Solid Cropped Basic Tube...,1799.0,Black,SASSAFRAS,http://assets.myntassets.com/assets/images/122...,657.0,4.079148,Black solid cropped basic jumpsuit with waist ...,{'Body or Garment Size': 'To-Fit Denotes Body ...,{'name': 'SASSAFRAS Women Black Solid Cropped ...,27
1,5445,495,17365646.0,I Saw It First Women Black & Yellow Floral Pri...,1299.0,Black,I Saw It First,http://assets.myntassets.com/assets/images/173...,NaN,NaN,Black and yellow floral printed A-line above t...,"{'Add-Ons': 'NA', 'Body Shape ID': '333,424', ...",{'name': 'I Saw It First Women Black & Yellow ...,29
2,2666,686,17570446.0,SHOWOFF Women Blue Boyfriend Fit High-Rise Low...,3720.0,Blue,SHOWOFF,http://assets.myntassets.com/assets/images/175...,NaN,NaN,"<ul> <li> Dark shade, light fade blue jeans <...","{'Add-Ons': 'NA', 'Body or Garment Size': 'To-...",{'name': 'SHOWOFF Women Blue Boyfriend Fit Hig...,48
3,5145,195,17829400.0,Forever New Pink Katerina Belted Tiered Skirt,5200.0,Pink,Forever New,http://assets.myntassets.com/assets/images/178...,NaN,NaN,<p></p><ul><li>A flowing romantic skirt to wea...,"{'Add-Ons': 'NA', 'Body Shape ID': '443,324,33...",{'name': 'Forever New Pink Katerina Belted Tie...,78
4,7754,324,16012536.0,Fabindia Blue Embroidered Tunic,2690.0,Blue,Fabindia,http://assets.myntassets.com/assets/images/160...,NaN,NaN,"Blue embroidered tunic ,has a round neck, thr...",{'Body or Garment Size': 'To-Fit Denotes Body ...,"{'name': 'Fabindia Blue Embroidered Tunic', 'c...",24


In [6]:
from langchain_community.document_loaders import DataFrameLoader

CSV Files

In [7]:
loader = DataFrameLoader(df, page_content_column="p_id")

In [8]:
loader.load()

[Document(page_content='12221918.0', metadata={'Unnamed: 0.1': 5975, 'Unnamed: 0': 35, 'name': 'SASSAFRAS Women Black Solid Cropped Basic Tube Jumpsuit', 'price': 1799.0, 'colour': 'Black', 'brand': 'SASSAFRAS', 'img': 'http://assets.myntassets.com/assets/images/12221918/2020/8/21/875980d0-5164-4789-a46f-0fe3f68108261598010237498-SASSAFRAS-Women-Black-Solid-Cropped-Basic-Tube-Jumpsuit-9815-1.jpg', 'ratingCount': 657.0, 'avg_rating': 4.079147640791477, 'description': "Black solid cropped basic jumpsuit with waist tie-up and side taping detail, strapless, cuffed hemsThe model (height 5'8'') is wearing a size SMaterial: Polyester<br>  Machine Wash", 'p_attributes': "{'Body or Garment Size': 'To-Fit Denotes Body Measurements in', 'Closure': 'NA', 'Fabric': 'Polyester', 'Neck': 'Strapless', 'Number of Pockets': 'NA', 'Pattern': 'Solid', 'Sleeve Length': 'Sleeveless', 'Surface Styling': 'Waist Tie-Ups', 'Type': 'Basic Jumpsuit', 'Wash Care': 'Machine Wash'}", 'Metadata': "{'name': 'SASSAFRAS

In [9]:
type(loader.load())

list

In [16]:
data=loader.load()

for d in data:
    print(d.metadata['description'])

SASSAFRAS Women Black Solid Cropped Basic Tube Jumpsuit
I Saw It First Women Black & Yellow Floral Print A-line Skirt
SHOWOFF Women Blue Boyfriend Fit High-Rise Low Distress Light Fade Stretchable Jeans
Forever New Pink Katerina Belted Tiered Skirt
Fabindia Blue Embroidered Tunic
ZOLA Women Black Mid-Rise Denim Shorts
AKISO Women Red Cropped Ethnic Palazzos
H&M Women Green Short Linen-Blend Skirt
Anekaant Women Black & Brown Woven-Design Woolen Shawl
Chhabra 555 Women Blue Dress Material
Geroo Jaipur Hand Crafted flared Red Pure Cotton Skirt with Blue Crop Top
Sangria Mustard Yellow Floral Print Kaftan Top
GRACIT Women Beige & Maroon Solid Flared Palazzos
Marie Claire Women Blue & Red Floral Print Cape Top
Saadgi Green & Gold-Toned Ethnic Motifs Embroidered Dupatta with Chikankari
Bhama Couture Women Black & Pink Yoke Design Pure Cotton High-Low Kurti
DressBerry Women Pink Solid Parallel Trousers
Kalt Women Charcoal
elleven Women Beige Original Trousers
Readiprint Fashions Women Grey &

In [21]:
os.environ["OPENAI_API_KEY"] = openai.api_key

In [22]:
# Import the OpenAI Embeddings class from LangChain
from langchain.embeddings import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings()

In [17]:
# Installing tiktoken library for working with OpenAI's embedding model
!pip install tiktoken

Defaulting to user installation because normal site-packages is not writeable


In [24]:
pip install faiss-cpu

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [25]:
# from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS
from langchain.vectorstores import Chroma

In [26]:
data

[Document(page_content='12221918.0', metadata={'Unnamed: 0.1': 5975, 'Unnamed: 0': 35, 'name': 'SASSAFRAS Women Black Solid Cropped Basic Tube Jumpsuit', 'price': 1799.0, 'colour': 'Black', 'brand': 'SASSAFRAS', 'img': 'http://assets.myntassets.com/assets/images/12221918/2020/8/21/875980d0-5164-4789-a46f-0fe3f68108261598010237498-SASSAFRAS-Women-Black-Solid-Cropped-Basic-Tube-Jumpsuit-9815-1.jpg', 'ratingCount': 657.0, 'avg_rating': 4.079147640791477, 'description': "Black solid cropped basic jumpsuit with waist tie-up and side taping detail, strapless, cuffed hemsThe model (height 5'8'') is wearing a size SMaterial: Polyester<br>  Machine Wash", 'p_attributes': "{'Body or Garment Size': 'To-Fit Denotes Body Measurements in', 'Closure': 'NA', 'Fabric': 'Polyester', 'Neck': 'Strapless', 'Number of Pockets': 'NA', 'Pattern': 'Solid', 'Sleeve Length': 'Sleeveless', 'Surface Styling': 'Waist Tie-Ups', 'Type': 'Basic Jumpsuit', 'Wash Care': 'Machine Wash'}", 'Metadata': "{'name': 'SASSAFRAS

In [27]:
# Store the split document in ChromaDB
db = Chroma.from_documents(data, OpenAIEmbeddings())

Perform Similarity Search

In [29]:
query = "find red kurta for me "
docs = db.similarity_search(query)
print(docs[0].metadata['description'])

Trendy and versatile, this playsuit is a worthy pick to add to your casualwear closet. This playsuit comes with an alluring ethnic motifs print and V-neck.&nbsp;<ul><li>Stylish black shade</li><li>Beautiful ethnic motifs print</li><li>V-neck</li><li>Sleeveless</li><li>Polyester, machine wash</li></ul><b>Trend Alert</b><br>Playsuits flaunt sophisticated and charming details, like knots, flared sleeves, plunge necklines, etc. in beautiful pastel palettes. It includes attractive prints and patterns, such as checks, floral, nautical stripes, and Nu Boheme, which enhance your overall appeal.The model (height 5'8) is wearing a size SPolyester<br>Machine wash<br>Wash dark colour separately<br>Do not bleach<br>Warm iron if needed


LangChain also support all major vector stores and databases such as FAISS, ElasticSearch, LanceDB, Milvus, Pinecone etc. Refer to the [API documentation](https://api.python.langchain.com/en/latest/api_reference.html#module-langchain.vectorstores) for the complete list.

In [30]:
from langchain.prompts import ChatPromptTemplate,SystemMessagePromptTemplate,HumanMessagePromptTemplate, PromptTemplate

In [48]:
system_template = """"
You are an AI fashion assistent which specializes in women fashion wear
that are affordable and nice for customer from the given set of {context}
You are also responsible for formatting the list response in a new line for every product 
and the data should contain these major informations 1) brand 2) dress description 3) price 
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_template="{product_request}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

Let's now create a simple LLMChain that takes an input, formats it, and passes it to an LLM for processing. The basic components are PromptTemplate, input queries, an LLM, and optional output parsers.

In [32]:
from langchain.chat_models import ChatOpenAI
llm_chat = ChatOpenAI()

C:\Users\apara\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [33]:
from langchain.llms import OpenAI
llm = OpenAI()

C:\Users\apara\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [38]:
from langchain_community.chat_models import ChatOpenAI
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

In [49]:

llm = ChatOpenAI(model_name="gpt-3.5-turbo")
chain = create_stuff_documents_chain(llm, chat_prompt)

In [56]:
query=input()

need a few red kurta


In [57]:
chain.invoke({"context": data,"product_request":query})

'Here are some red kurta options for you:\n\n1) Brand: 17365646.0\n   Description: Red Cotton Kurta\n   Price: 17365646.0\n\n2) Brand: 17570446.0\n   Description: Red Silk Kurta with Golden Embroidery\n   Price: 17570446.0\n\n3) Brand: 17829400.0\n   Description: Red Linen Kurta with Floral Print\n   Price: 17829400.0\n\n4) Brand: 16012536.0\n   Description: Red Chanderi Kurta with Zari Work\n   Price: 16012536.0\n\n5) Brand: 14579962.0\n   Description: Red Crepe Kurta with Sequin Detailing\n   Price: 14579962.0\n\nFeel free to let me know if you need more options or details on any specific product!'

In [58]:
query=input()
chain.invoke({"context": data,"product_request":query})

jeans please


'Here are some affordable and stylish jeans for you:\n\n1) Brand: 17365646.0\n   Dress Description: Skinny High Ankle Jeans\n   Price: 17365646.0\n\n2) Brand: 17570446.0\n   Dress Description: Slim Fit Mid-Rise Jeans\n   Price: 17570446.0\n\n3) Brand: 17829400.0\n   Dress Description: Bootcut Jeans\n   Price: 17829400.0\n\n4) Brand: 16012536.0\n   Dress Description: Distressed Boyfriend Jeans\n   Price: 16012536.0\n\n5) Brand: 14579962.0\n   Dress Description: Flare Leg Jeans\n   Price: 14579962.0\n\n6) Brand: 18534256.0\n   Dress Description: High Waisted Skinny Jeans\n   Price: 18534256.0\n\n7) Brand: 19147756.0\n   Dress Description: Cropped Straight Leg Jeans\n   Price: 19147756.0\n\n8) Brand: 16196158.0\n   Dress Description: Wide Leg Jeans\n   Price: 16196158.0\n\n9) Brand: 18780564.0\n   Dress Description: Mom Jeans\n   Price: 18780564.0\n\n10) Brand: 12101936.0\n    Dress Description: Distressed Skinny Jeans\n    Price: 12101936.0\n\nFeel free to let me know if you need more op

In [59]:

query=input()
chain.invoke({"context": data,"product_request":query})

I am looking for white tops for women


'Here are some white tops for women that you might like:\n\n1) Brand: 17365646.0\n   Dress Description: White Button-Up Blouse\n   Price: $49.99\n\n2) Brand: 18534256.0\n   Dress Description: White Off-Shoulder Top\n   Price: $29.99\n\n3) Brand: 19147756.0\n   Dress Description: White Ruffle Sleeve Blouse\n   Price: $39.99\n\n4) Brand: 17158526.0\n   Dress Description: White Lace Trim Tank Top\n   Price: $19.99\n\n5) Brand: 18004088.0\n   Dress Description: White Peplum Hem Blouse\n   Price: $34.99\n\n6) Brand: 18003804.0\n   Dress Description: White Crochet Detail Top\n   Price: $27.99\n\n7) Brand: 17931206.0\n   Dress Description: White V-Neck Tunic\n   Price: $22.99\n\n8) Brand: 17765672.0\n   Dress Description: White Cold Shoulder Top\n   Price: $31.99\n\n9) Brand: 18068668.0\n   Dress Description: White Tie-Front Blouse\n   Price: $25.99\n\n10) Brand: 18813862.0\n    Dress Description: White Embroidered Peasant Top\n    Price: $37.99\n\nFeel free to ask if you need more informatio